In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [5]:
locH_a1_temps = pd.read_excel("Country H.xlsx", sheet_name=1, skiprows=4).dropna().reset_index(drop=True)
locH_a1_temps['Date'] = datetime(1970,1,1)
locH_a1_temps['Avg_Temp'] = np.nan
for i, row in locH_a1_temps.iterrows():
    yr, mt, d, maxt, mint, dt, avgt = row.values
    locH_a1_temps.loc[i, 'Date'] = datetime(int(yr), int(mt), int(d))
    locH_a1_temps.loc[i, 'Avg_Temp'] = (maxt + mint)/2
locH_a1_temps['doy'] = locH_a1_temps.Date.apply(lambda x: x.day_of_year)
locH_a1_temps = locH_a1_temps.rename(columns={'Year':'year', 'max ':'Max_Temp', 'min':'Min_Temp'}).drop("Day",axis=1)
locH_a1_temps

,year,Month,Max_Temp,Min_Temp,Date,Avg_Temp,doy
0,1991.0,1.0,29.2,23.700000,1991-01-01,26.450000,1
1,1991.0,1.0,29.2,23.000000,1991-01-02,26.100000,2
2,1991.0,1.0,29.0,23.800000,1991-01-03,26.400000,3
3,1991.0,1.0,29.2,23.400000,1991-01-04,26.300000,4
4,1991.0,1.0,29.3,23.300000,1991-01-05,26.300000,5
...,...,...,...,...,...,...,...
10953,2020.0,12.0,30.3,26.208333,2020-12-27,28.254167,362
10954,2020.0,12.0,29.6,26.441667,2020-12-28,28.020833,363
10955,2020.0,12.0,30.6,26.204167,2020-12-29,28.402083,364
10956,2020.0,12.0,30.0,25.204167,2020-12-30,27.602083,365


In [8]:
locH_a1_rain = pd.read_excel("Country H.xlsx", sheet_name=2, skiprows=3).dropna().reset_index(drop=True)
locH_a1_rain['Date'] = datetime(1970,1,1)
for i, row in locH_a1_rain.iterrows():
    yr, mt, d, rain, dt = row.values
    locH_a1_rain.loc[i, 'Date'] = datetime(int(yr), int(mt), int(d))
locH_a1_rain

,Year,Month,Day,Rainfall,Date
0,1991,1,1,10,1991-01-01
1,1991,1,2,3.5,1991-01-02
2,1991,1,3,9,1991-01-03
3,1991,1,4,1,1991-01-04
4,1991,1,5,1.3,1991-01-05
...,...,...,...,...,...
10953,2020,12,27,15.8,2020-12-27
10954,2020,12,28,T,2020-12-28
10955,2020,12,29,T,2020-12-29
10956,2020,12,30,1.2,2020-12-30


In [20]:
df = locH_a1_temps.merge(locH_a1_rain[['Rainfall']], left_index=True, right_index=True, how='left')
df['year'] = df.year.astype(int)
df['Month'] = df.Month.astype(int)
df['area'] = 1
df['country'] = "H"
df = df.rename(columns={'Rainfall':"rain"})[["doy","Month","year","rain","Date","Avg_Temp","Min_Temp","Max_Temp","area","country"]]
df['rain'] = df.rain.apply(lambda x: float(x) if x not in ["M", "T"] else np.nan)
df

,doy,Month,year,rain,Date,Avg_Temp,Min_Temp,Max_Temp,area,country
0,1,1,1991,10.0,1991-01-01,26.450000,23.700000,29.2,1,H
1,2,1,1991,3.5,1991-01-02,26.100000,23.000000,29.2,1,H
2,3,1,1991,9.0,1991-01-03,26.400000,23.800000,29.0,1,H
3,4,1,1991,1.0,1991-01-04,26.300000,23.400000,29.2,1,H
4,5,1,1991,1.3,1991-01-05,26.300000,23.300000,29.3,1,H
...,...,...,...,...,...,...,...,...,...,...
10953,362,12,2020,15.8,2020-12-27,28.254167,26.208333,30.3,1,H
10954,363,12,2020,NaN,2020-12-28,28.020833,26.441667,29.6,1,H
10955,364,12,2020,NaN,2020-12-29,28.402083,26.204167,30.6,1,H
10956,365,12,2020,1.2,2020-12-30,27.602083,25.204167,30.0,1,H


In [21]:
df.to_parquet("CountryH_Area1_cleaned_merged.parquet")